In [ ]:
!pip install opencv-python

In [ ]:
# imported image


import cv2
face_cascade = cv2.CascadeClassifier (cv2.data.haarcascades + 'haarcascade_frontalface_default.xml') 
image_path= 'C:/Users/TANYA/OneDrive/Desktop/PRACTICALS/facedetection_input_img.jpg'
image=cv2.imread(image_path)
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
faces= face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30)) 
for ( x, y, w, h) in faces:
    cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)
    
output_path = 'C:/Users/TANYA/OneDrive/Desktop/PRACTICALS/faces_detected.jpg'
cv2.imwrite(output_path, image)
print(f"Faces detected: {len(faces)}, Output saved to {output_path}")

In [ ]:
# image from camera


import cv2
import matplotlib.pyplot as plt
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml') 
cap= cv2.VideoCapture(0)
ret, frame = cap.read()
if ret:
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale (gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30)) 
    for ( x, y, W, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2) 
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show() 
cap.release()
print("Number of faces detected:", len(faces))

In [ ]:
# video


import cv2
import matplotlib.pyplot as plt
pedestrian_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_fullbody.xml')
video_path = 'C:/Users/TANYA/OneDrive/Desktop/PRACTICALS/video.mp4'
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: Unable to open the video")
    exit()
while True:
    ret, frame = cap.read()
    if not ret:
        break   
    gray_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    # Detect pedestrians in the grayscale frame
    pedestrians = pedestrian_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30,30))
    # Draw rectangle around the detected pedestrians
    for (x,y,w,h) in pedestrians:
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    plt.axis('off')   # Turn off the axis
    plt.show()
cap.release()

In [ ]:
# stitched


import cv2
import numpy as np
import matplotlib.pyplot as plt

img_ = cv2.imread('C:/Users/TANYA/OneDrive/Desktop/PRACTICALS/right.jpg')
img1 = cv2.cvtColor(img_, cv2.COLOR_BGR2GRAY)
img = cv2.imread('C:/Users/TANYA/OneDrive/Desktop/PRACTICALS/left.jpg')
img2 = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

sift = cv2.SIFT_create()
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)

matches = bf.knnMatch(des1, des2, k=2)

good = []
for m, n in matches:
    if m.distance < 0.75 * n.distance:
        good.append(m)
        
if len(good) > 4:
    
    src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
    
    H, status = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    
    dst = cv2.warpPerspective(img_, H, (img.shape[1] + img_.shape[1], img.shape[0]))
    
    dst_rgb = cv2.cvtColor(dst, cv2.COLOR_BGR2RGB)
    
    plt.subplot(122), plt.imshow(dst_rgb), plt.title('Warped Image')
    plt.show()
    
    dst[0:img.shape[0], 0:img.shape[1]] = img
    
    combined_rgb = cv2.cvtColor(dst, cv2.COLOR_BGR2RGB)
    
    cv2.imwrite('output.jpg', dst)
    
    plt.imshow(combined_rgb)
    plt.title('Stitched Image')
    plt.show()
else:
    raise AssertionError("Not enough matches are found - {}/{}".format(len(good), 4))